In [267]:
import numpy as np

# MDPの構築
from typing import NamedTuple ,Optional
np.random.seed(10)

S = 5# 状態数
A = 2 # 行動数
S_set = np.arange(S)
A_set = np.arange(A)
gamma = 0.9 # 割引率

rew = np.random.uniform(0,1,size=(S,A)) # 報酬
rew = np.array(rew)

P = np.random.rand(S,A,S) # 遷移確率
P = P.reshape(S*A,S)
P = P/np.sum(P,axis=1,keepdims=True) # 正規化
P = P.reshape(S,A,S)
np.testing.assert_allclose(P.sum(axis=-1), 1, atol=1e-6)

class MDP(NamedTuple):
    S_set: np.ndarray
    A_set: np.ndarray
    rew: np.ndarray
    P: np.ndarray
    gamma: float
    H: int
    K:int


    optimal_V: Optional[np.ndarray] = None

    @property
    def S(self):
        return len(self.S_set)

    @property
    def A(self):
        return len(self.A_set)

H = int (1/(1-gamma) + 20)
mdp = MDP(S_set,A_set,rew,P,gamma,H,K=50)

In [268]:
def sampler(mdp: MDP, policy,s,h):
    """状態sで行動aをとり、次の状態と報酬を返す"""
    a = policy[h,s]
    a = int(a)
    # print(a)
    next_s = np.random.choice(mdp.S_set, p=mdp.P[s, a])
    rew = mdp.rew[s, a]
    return next_s, rew

In [269]:
def feature_func(n:int,delta:float,mdp:MDP):
    if n == 0:
        return 1
    return np.min([1,np.sqrt((0.52/n) * (1.4*np.log(np.log(np.max([np.e,n]))) + np.log(26*mdp.S*mdp.A*(mdp.H+1+mdp.S)/delta)))])

In [270]:
def V_h_max(h:int,mdp:MDP):
    return (mdp.H-h+1)

In [271]:
def V_std(P,V):
    # return np.sqrt(np.sum(P*V**2,axis=-1) - np.sum(P*V,axis=-1)**2)
    # return np.sqrt(np.sum(P*(V-P*V)**2,axis=-1))
    # std = np.sqrt(np.sum(P*V**2,axis=-1) - np.sum(2*(P**2) * V**2,axis=-1) + np.sum(P**3 * V**2,axis=-1))
    PV = np.sum(P*V,axis=-1)
    std = np.sqrt(np.sum(P * (V-PV)**2,axis=-1))
    return std

In [272]:
def ORLC(mdp:MDP,delta:float):
    n_k = np.zeros((mdp.S,mdp.A))
    n_k_p = np.zeros((mdp.S,mdp.A,mdp.S))
    r_k = np.zeros((mdp.S,mdp.A))
    P_k = np.zeros((mdp.S,mdp.A,mdp.S))
    V_lower = np.zeros((mdp.K,mdp.H,mdp.S))
    Q_lower = np.zeros((mdp.K,mdp.H-1,mdp.S,mdp.A))
    V_upper = np.zeros((mdp.K,mdp.H,mdp.S))
    Q_upper = np.zeros((mdp.K,mdp.H-1,mdp.S,mdp.A))
    gzi = np.zeros((mdp.K,mdp.H,mdp.S,mdp.A))
    policy = np.zeros((mdp.K,mdp.H,mdp.S))
    epsilon = np.zeros(mdp.K)
    experience = []
    for k in range(mdp.K):
        if k != 0:
            for h in range(mdp.H):
                s, a, r, s_dash = experience[h]
                n_k[s, a] += 1
                n_k_p[s, a, s_dash] += 1
                r_k[s, a] += r
                P_k[s, a, s_dash] = n_k_p[s, a, s_dash] / n_k[s, a]
        for h in reversed(range(mdp.H)):
            for s in range(mdp.S):
                for a in range(mdp.A):
                    # print(V_std(P_k[s,a],V_upper[k,h]))
                    gzi[k,h-1,s,a] = (1 + np.sqrt(12)*V_std(P_k[s,a],V_upper[k,h,s]) * feature_func(n_k[s,a],delta,mdp)) + 45*mdp.S*mdp.H**2 * feature_func(n_k[s,a],delta,mdp) **2 + 1/mdp.H *np.sum(P_k[s,a]*(V_upper[k,h] - V_lower[k,h]))
                    Q_upper[k,h-1,s,a] = np.max([0,r_k[s,a] + np.sum(P_k[s,a,:]*V_upper[k,h,:]) + gzi[k,h-1,s,a]])
                    Q_upper[k,h-1,s,a] = np.min([V_h_max(h,mdp),Q_upper[k,h-1,s,a]])
                    Q_lower[k,h-1,s,a] = np.max([0,r_k[s,a] + np.sum(P_k[s,a]*V_lower[k,h]) - gzi[k,h-1,s,a]])
                    Q_lower[k,h-1,s,a] = np.min([V_h_max(h,mdp),Q_lower[k,h-1,s,a]])
                    
        for h in range(mdp.H):
            for s in range(mdp.S):
                policy[k,h-1,s] = np.argmax(Q_upper[k,h-1,s])
                a = policy[k,h-1,s]
                a = int(a)
                V_upper[k,h-1,s] = Q_upper[k,h-1,s,a]
                V_lower[k,h-1,s] = Q_lower[k,h-1,s,a]
                
        epsilon[k] = np.abs(V_upper[k,0,0] - V_lower[k,0,0])
        s = 0
        experience = []
        for h in range(mdp.H):
            s_dash,rew = sampler(mdp,policy[k],s,h)
            experience.append([s,int(policy[k,h,s]),rew,s_dash])
            s=s_dash
    return policy,epsilon,Q_upper,Q_lower,V_upper,V_lower

In [273]:
def ORLC(mdp:MDP,delta:float):
    n_k = np.zeros((mdp.S,mdp.A))
    n_k_p = np.zeros((mdp.S,mdp.A,mdp.S))
    r_hat_k = np.zeros((mdp.S,mdp.A))
    r_k = np.zeros((mdp.S,mdp.A))
    P_k = np.zeros((mdp.S,mdp.A,mdp.S))
    V_lower = np.ones((mdp.H,mdp.S))
    Q_lower = np.ones((mdp.H-1,mdp.S,mdp.A))
    V_upper = np.ones((mdp.H,mdp.S))
    Q_upper = np.ones((mdp.H-1,mdp.S,mdp.A))
    gzi = np.zeros((mdp.H,mdp.S,mdp.A))
    policy = np.zeros((mdp.H,mdp.S))
    epsilon = np.zeros(mdp.K)
    experience = []
    for k in range(mdp.K):
        if k != 0:
            for h in range(mdp.H):
                s, a, r, s_dash = experience[h]
                n_k[s, a] += 1
                n_k_p[s, a, s_dash] += 1
                r_hat_k[s, a] += r
                r_k[s,a] = r_hat_k[s,a] / n_k[s,a]
                # P_k[s, a, s_dash] = n_k_p[s, a, s_dash] / n_k[s, a]
                P_k[s,a,:] = n_k_p[s,a,:] / n_k[s,a]
        for h in reversed(range(mdp.H)):
            for s in range(mdp.S):
                for a in range(mdp.A):
                    # print(V_std(P_k[s,a],V_upper[k,h]))
                    gzi[h-1,s,a] = (1 + np.sqrt(12)*V_std(P_k[s,a],V_upper[h,s]) * feature_func(n_k[s,a],delta,mdp)) + 45*mdp.S*mdp.H**2 * feature_func(n_k[s,a],delta,mdp) **2 + 1/mdp.H *np.sum(P_k[s,a]*(V_upper[h] - V_lower[h]))
                    # print(feature_func(n_k[s,a],delta,mdp))
                    # gzi[h-1,s,a] = (1 + np.sqrt(12)*V_std(P_k[s,a],V_upper[h,s]) * feature_func(n_k[s,a],delta,mdp))+1/mdp.H *np.sum(P_k[s,a]*(V_upper[h] - V_lower[h]))
                    Q_upper[h-1,s,a] = np.max([0,r_k[s,a] + np.sum(P_k[s,a,:]*V_upper[h,:]) + gzi[h-1,s,a]])
                    print(feature_func(n_k[s,a],delta,mdp))
                    print(Q_upper[h-1,s,a])
                    Q_upper[h-1,s,a] = np.min([V_h_max(h,mdp),Q_upper[h-1,s,a]])
                    print(Q_upper[h-1,s,a])
                    print('------')
                    Q_lower[h-1,s,a] = np.max([0,r_k[s,a] + np.sum(P_k[s,a]*V_lower[h]) - gzi[h-1,s,a]])
                    Q_lower[h-1,s,a] = np.min([V_h_max(h,mdp),Q_lower[h-1,s,a]])
        for h in range(mdp.H):
            for s in range(mdp.S):
                policy[h-1,s] = np.argmax(Q_upper[h-1,s],axis=-1)
                a = policy[h-1,s]
                a = int(a)
                V_upper[h-1,s] = Q_upper[h-1,s,a]
                V_lower[h-1,s] = Q_lower[h-1,s,a]
        epsilon[k] = np.abs(V_upper[0,0] - V_lower[0,0])
        s = 0
        experience = []
        for h in range(mdp.H):
            s_dash,rew = sampler(mdp,policy,s,h)
            experience.append([s,int(policy[h,s]),rew,s_dash])
            s=s_dash
    return policy,epsilon,Q_upper,Q_lower,V_upper,V_lower,r_k,P_k

In [274]:
def vector_1_norm(V1, V2):
    """
    Calculate the 1-norm of the difference between two vectors V1 and V2.

    :param V1: First vector.
    :param V2: Second vector.
    :return: The 1-norm of the difference between V1 and V2.
    """
    return np.sum(np.abs(V1 - V2))

In [275]:
def gzi_upper_(mdp:MDP,h:int,feature_func,n_k,s,a,delta,P_k,V_upper,V_lower):
    gzi1 = (V_h_max(h,mdp) + 1) *feature_func(n_k[s,a],delta,mdp)
    gzi2 = (1 + np.sqrt(12) * np.sqrt(V_std(P_k[s,a],V_upper[h]) ** 2 +np.sum(P_k[s,a] * (V_upper[h] - V_lower[h]) ** 2,axis=-1)) * feature_func(n_k[s,a],delta,mdp)) + 8.13*V_h_max(h,mdp) * feature_func(n_k[s,a],delta,mdp) ** 2
    gzi3 = (1 + np.sqrt(12) * V_std(P_k[s,a],V_upper[h])) * feature_func(n_k[s,a],delta,mdp) + 1/mdp.H * np.sum(P_k[s,a] * (V_upper[h] - V_lower[h])) + (20.13 * mdp.H * vector_1_norm(V_upper[h],V_lower[h]) * feature_func(n_k[s,a],delta,mdp)) ** 2
    gzi = np.min([gzi1,gzi2,gzi3])
    # print(gzi1)
    # print(gzi2)
    # print(gzi3)
    # print(gzi)
    # print('---')
    return gzi

In [276]:
def ORLC(mdp:MDP,delta:float):
    np.random.seed(10)
    n_k = np.zeros((mdp.S,mdp.A))
    n_k_p = np.zeros((mdp.S,mdp.A,mdp.S))
    r_hat_k = np.zeros((mdp.S,mdp.A))
    r_k = np.zeros((mdp.S,mdp.A))
    P_k = np.zeros((mdp.S,mdp.A,mdp.S))
    V_lower = np.ones((mdp.H,mdp.S))
    Q_lower = np.ones((mdp.H-1,mdp.S,mdp.A))
    V_upper = np.ones((mdp.H,mdp.S))
    Q_upper = np.ones((mdp.H-1,mdp.S,mdp.A))
    gzi = np.zeros((mdp.H,mdp.S,mdp.A))
    policy = np.zeros((mdp.H,mdp.S))
    epsilon = np.zeros(mdp.K)
    experience = []
    for k in range(mdp.K):
        if k != 0:
            for h in range(mdp.H):
                s, a, r, s_dash = experience[h]
                n_k[s, a] += 1
                n_k_p[s, a, s_dash] += 1
                r_hat_k[s, a] += r
                r_k[s,a] = r_hat_k[s,a] / n_k[s,a]
                # P_k[s, a, s_dash] = n_k_p[s, a, s_dash] / n_k[s, a]
                P_k[s,a,:] = n_k_p[s,a,:] / n_k[s,a]
        for h in reversed(range(mdp.H)):
            for s in range(mdp.S):
                for a in range(mdp.A):
                    # print(V_std(P_k[s,a],V_upper[k,h]))
                    gzi[h-1,s,a] = (1 + np.sqrt(12)*V_std(P_k[s,a],V_upper[h,s]) * feature_func(n_k[s,a],delta,mdp)) + 45*mdp.S*mdp.H**2 * feature_func(n_k[s,a],delta,mdp) **2 + 1/mdp.H *np.sum(P_k[s,a]*(V_upper[h] - V_lower[h]))
                    # print(feature_func(n_k[s,a],delta,mdp))
                    # gzi[h-1,s,a] = (1 + np.sqrt(12)*V_std(P_k[s,a],V_upper[h,s]) * feature_func(n_k[s,a],delta,mdp))+1/mdp.H *np.sum(P_k[s,a]*(V_upper[h] - V_lower[h]))
                    gzi_upper = gzi_upper_(mdp,h,feature_func,n_k,s,a,delta,P_k,V_upper,V_lower)
                    Q_upper[h-1,s,a] = np.max([0,r_k[s,a] + np.sum(P_k[s,a,:]*V_upper[h,:]) + gzi_upper])
                    # print(feature_func(n_k[s,a],delta,mdp))
                    # print(Q_upper[h-1,s,a])
                    Q_upper[h-1,s,a] = np.min([V_h_max(h-1,mdp),Q_upper[h-1,s,a]])
                    # print(Q_upper[h-1,s,a])
                    # print('------')
                    Q_lower[h-1,s,a] = np.max([0,r_k[s,a] + np.sum(P_k[s,a]*V_lower[h]) - gzi[h-1,s,a]])
                    Q_lower[h-1,s,a] = np.min([V_h_max(h-1,mdp),Q_lower[h-1,s,a]])
        for h in range(mdp.H):
            for s in range(mdp.S):
                # policy[h-1,s] = np.argmax(Q_upper[h-1,s],axis=-1)
                max_value = np.max(Q_upper[h-1,s],axis=-1)
                max_indeices = [i for i,value in enumerate(Q_upper[h-1,s]) if value == max_value]
                a = np.random.choice(max_indeices)
                policy[h-1,s] = a
                print(policy[h-1,s],a)
                V_upper[h-1,s] = Q_upper[h-1,s,a]
                V_lower[h-1,s] = Q_lower[h-1,s,a]
        epsilon[k] = np.abs(V_upper[0,0] - V_lower[0,0])
        s = 0
        experience = []
        for h in range(mdp.H):
            s_dash,rew = sampler(mdp,policy,s,h)
            experience.append([s,int(policy[h,s]),rew,s_dash])
            s=s_dash
    return policy,epsilon,Q_upper,Q_lower,V_upper,V_lower,r_k,P_k

In [277]:
a = np.array([1,2,3,4,5])
a.argmax()

4

In [278]:
policy,epsilon,Qu,Ql,Vu,Vl,r,P = ORLC(mdp,10)

1.0 1
1.0 1
0.0 0
1.0 1
0.0 0
1.0 1
1.0 1
0.0 0
1.0 1
1.0 1
0.0 0
1.0 1
1.0 1
0.0 0
0.0 0
1.0 1
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
1.0 1
0.0 0
0.0 0
1.0 1
1.0 1
0.0 0
0.0 0
1.0 1
0.0 0
0.0 0
1.0 1
0.0 0
0.0 0
0.0 0
1.0 1
1.0 1
0.0 0
1.0 1
1.0 1
1.0 1
1.0 1
1.0 1
0.0 0
1.0 1
0.0 0
0.0 0
0.0 0
0.0 0
1.0 1
0.0 0
1.0 1
1.0 1
1.0 1
0.0 0
1.0 1
0.0 0
1.0 1
1.0 1
0.0 0
1.0 1
0.0 0
0.0 0
1.0 1
0.0 0
0.0 0
0.0 0
1.0 1
1.0 1
0.0 0
0.0 0
0.0 0
1.0 1
0.0 0
1.0 1
1.0 1
0.0 0
1.0 1
1.0 1
0.0 0
1.0 1
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
1.0 1
0.0 0
1.0 1
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
1.0 1
1.0 1
0.0 0
1.0 1
1.0 1
0.0 0
0.0 0
1.0 1
0.0 0
0.0 0
1.0 1
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
1.0 1
1.0 1
1.0 1
0.0 0
0.0 0
0.0 0
1.0 1
1.0 1
1.0 1
1.0 1
0.0 0
1.0 1
0.0 0
1.0 1
1.0 1
1.0 1
0.0 0
0.0 0
1.0 1
1.0 1
1.0 1
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
1.0 1
1.0 1
1.0 1
1.0 1
0.0 0
1.0 1
1.0 1
0.0 0
0.0 0
0.0 0
1.0 1
0.0 0
1.0 1
1.0 1
0.0 0
0.0 0
1.0 1
1.0 1
1.0 1
0.0 0
1.0 1
1.0 1
1.0 1
1.0 1
0.0 0
1.0 1
1.0 1
0.0 

In [279]:
Qu

array([[[31.        , 31.        ],
        [31.        , 31.        ],
        [31.        , 31.        ],
        [31.        , 31.        ],
        [31.        , 31.        ]],

       [[30.        , 30.        ],
        [30.        , 30.        ],
        [30.        , 30.        ],
        [30.        , 30.        ],
        [30.        , 30.        ]],

       [[29.        , 29.        ],
        [29.        , 29.        ],
        [29.        , 29.        ],
        [29.        , 29.        ],
        [29.        , 29.        ]],

       [[28.        , 28.        ],
        [28.        , 28.        ],
        [28.        , 28.        ],
        [28.        , 28.        ],
        [28.        , 28.        ]],

       [[27.        , 27.        ],
        [27.        , 27.        ],
        [27.        , 27.        ],
        [27.        , 27.        ],
        [27.        , 27.        ]],

       [[26.        , 26.        ],
        [26.        , 26.        ],
        [26.      

In [280]:
epsilon

array([ 1., 31., 31., 31., 31., 31., 31., 31., 31., 31., 31., 31., 31.,
       31., 31., 31., 31., 31., 31., 31., 31., 31., 31., 31., 31., 31.,
       31., 31., 31., 31., 31., 31., 31., 31., 31., 31., 31., 31., 31.,
       31., 31., 31., 31., 31., 31., 31., 31., 31., 31., 31.])

In [281]:
P

array([[[0.295     , 0.32      , 0.        , 0.155     , 0.23      ],
        [0.20540541, 0.22162162, 0.08108108, 0.24864865, 0.24324324]],

       [[0.31543624, 0.08724832, 0.16107383, 0.22147651, 0.2147651 ],
        [0.17931034, 0.26206897, 0.19310345, 0.22068966, 0.14482759]],

       [[0.26973684, 0.18421053, 0.39473684, 0.13157895, 0.01973684],
        [0.14583333, 0.02083333, 0.54166667, 0.00694444, 0.28472222]],

       [[0.17361111, 0.3125    , 0.04166667, 0.375     , 0.09722222],
        [0.31932773, 0.16806723, 0.28571429, 0.15966387, 0.06722689]],

       [[0.23140496, 0.04132231, 0.44628099, 0.08264463, 0.19834711],
        [0.24324324, 0.3963964 , 0.09009009, 0.23423423, 0.03603604]]])

In [282]:
mdp.P

array([[[0.23095415, 0.32127671, 0.0013305 , 0.17259974, 0.2738389 ],
        [0.1879775 , 0.22149875, 0.08957355, 0.28165476, 0.21929543]],

       [[0.24955791, 0.06539495, 0.17172815, 0.31008593, 0.20323308],
        [0.15410484, 0.2193498 , 0.1821994 , 0.23093577, 0.2134102 ]],

       [[0.3044075 , 0.19720409, 0.34350667, 0.12068438, 0.03419736],
        [0.1568966 , 0.05947375, 0.43238197, 0.02446914, 0.32677854]],

       [[0.19737645, 0.29531053, 0.07171029, 0.30885016, 0.12675257],
        [0.3111855 , 0.12204237, 0.36449885, 0.13422754, 0.06804573]],

       [[0.21305764, 0.05072857, 0.44568102, 0.08204253, 0.20849024],
        [0.27245984, 0.28497244, 0.13166349, 0.23837198, 0.07253225]]])

In [240]:
policy[108]

array([1., 1., 0., 1., 0.])

In [241]:
def Q_iteration(mdp:MDP):
    Q = np.zeros((mdp.S,mdp.A))
    for s in range(mdp.S):
        for a in range(mdp.A):
            Q[s,a] = mdp.rew[s,a] + mdp.gamma * np.sum(mdp.P[s,a,:] * np.max(Q,axis=-1))
    return Q
Q = Q_iteration(mdp)
a_index = np.argmax(Q,axis=-1)



In [242]:
a_index

array([0, 1, 0, 1, 1])